## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [1]:
import random
import numpy as np
import pandas as pd
import string
from collections import defaultdict

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [3]:
from random import randint
N = 1000000
A = [randint(0, 1000) for _ in range(N)]
B = [a + 100 for a in A] 
Average_B = sum(B) / N
print(Average_B)

599.441979


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [4]:
n_strok = 2000000
n_col = 4

data = [[random.random() for i in range(n_col)] for i in range(n_strok)]
dtfr = pd.DataFrame(data, columns = ["Столбец 1", "Столбец 2", "Столбец 3", "Столбец 4"])

letters = list(string.ascii_uppercase)
key_column = random.choices(letters, k = n_strok)
dtfr["key"] = key_column

podmnozhestvo = dtfr[dtfr["key"].isin(letters[:5])]
print(podmnozhestvo)

         Столбец 1  Столбец 2  Столбец 3  Столбец 4 key
1         0.324637   0.855391   0.643027   0.633980   C
3         0.731551   0.211427   0.494211   0.407324   E
17        0.148254   0.729788   0.611118   0.127282   C
18        0.984990   0.103573   0.499724   0.284710   E
21        0.096050   0.837872   0.955175   0.089940   A
...            ...        ...        ...        ...  ..
1999981   0.723191   0.723890   0.230369   0.209599   A
1999990   0.513640   0.904358   0.255626   0.283254   C
1999992   0.197475   0.773648   0.815148   0.313554   B
1999993   0.918365   0.943757   0.610022   0.800675   C
1999994   0.516663   0.757562   0.046410   0.204297   E

[383554 rows x 5 columns]


## Лабораторная работа 3

In [26]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [9]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')

recipes['submitted'] = pd.to_datetime(recipes['submitted'])
reviews['date'] = pd.to_datetime(reviews['date'])

In [13]:
def calculate_mean_rating_a(df):
    total_rating = 0
    count = 0
    
    for _, row in df.iterrows():
        if row['date'].year == 2010:
            total_rating += row['rating']
            count += 1
    
    if count == 0:
        return 0
    
    return total_rating / count

mean_rating_a = calculate_mean_rating_a(reviews)
print("Среднее значение рейтинга (метод A):", mean_rating_a)

Среднее значение рейтинга (метод A): 4.4544402182900615


In [14]:
%%time
calculate_mean_rating_a(reviews)

CPU times: user 6.4 s, sys: 30.7 ms, total: 6.43 s
Wall time: 6.54 s


4.4544402182900615

In [15]:
def calculate_mean_rating_b(df):
    total_rating = 0
    count = 0
    
    for _, row in df[df['date'].dt.year == 2010].iterrows():
        total_rating += row['rating']
        count += 1
    
    if count == 0:
        return 0
    
    return total_rating / count

mean_rating_b = calculate_mean_rating_b(reviews)
print("Среднее значение рейтинга (метод B):", mean_rating_b)

Среднее значение рейтинга (метод B): 4.4544402182900615


In [16]:
%%time
calculate_mean_rating_b(reviews)

CPU times: user 777 ms, sys: 605 µs, total: 778 ms
Wall time: 813 ms


4.4544402182900615

In [17]:
def calculate_mean_rating_c(df):
    mean_rating = df[df['date'].dt.year == 2010]['rating'].mean()
    
    if pd.isnull(mean_rating):
        return 0
    
    return mean_rating

mean_rating_c = calculate_mean_rating_c(reviews)
print("Среднее значение рейтинга (метод C):", mean_rating_c)

Среднее значение рейтинга (метод C): 4.4544402182900615


In [18]:
%%time
calculate_mean_rating_c(reviews)

CPU times: user 22.2 ms, sys: 970 µs, total: 23.1 ms
Wall time: 28.1 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [28]:
import line_profiler
profiler = line_profiler.LineProfiler(calculate_mean_rating_a, calculate_mean_rating_b, calculate_mean_rating_c)
profiler.enable()

mean_rating_a = calculate_mean_rating_a(reviews)
mean_rating_b = calculate_mean_rating_b(reviews)
mean_rating_c = calculate_mean_rating_c(reviews)

profiler.disable()
profiler.print_stats()

Timer unit: 1e-09 s

Total time: 11.6136 s
File: <ipython-input-13-e660e5f5c04b>
Function: calculate_mean_rating_a at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calculate_mean_rating_a(df):
     2         1       1877.0   1877.0      0.0      total_rating = 0
     3         1        273.0    273.0      0.0      count = 0
     4                                               
     5    126696 10085176208.0  79601.4     86.8      for _, row in df.iterrows():
     6    114602 1410947010.0  12311.7     12.1          if row['date'].year == 2010:
     7     12094  112368209.0   9291.2      1.0              total_rating += row['rating']
     8     12094    5103236.0    422.0      0.0              count += 1
     9                                               
    10         1        347.0    347.0      0.0      if count == 0:
    11                                                   return 0
    12                

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [29]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [30]:
import cProfile

def profile_get_word_reviews_count():
    cProfile.run('get_word_reviews_count(reviews)', sort='tottime')

profile_get_word_reviews_count()

         52986037 function calls (51972523 primitive calls) in 49.753 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   11.613   11.613   49.620   49.620 <ipython-input-29-b1bc049bcd0c>:1(get_word_reviews_count)
   253364    2.862    0.000   25.546    0.000 series.py:342(__init__)
10846018/9832522    1.990    0.000    2.408    0.000 {built-in method builtins.len}
8995349/8995343    1.882    0.000    2.757    0.000 {built-in method builtins.isinstance}
   253364    1.613    0.000    7.110    0.000 construction.py:493(sanitize_array)
   253376    1.577    0.000    3.792    0.000 generic.py:5904(__setattr__)
  6792020    1.372    0.000    1.372    0.000 {method 'append' of 'list' objects}
   506720    1.237    0.000    1.237    0.000 {method 'split' of 'str' objects}
   253372    1.203    0.000    1.846    0.000 generic.py:5844(__finalize__)
   380037    1.200    0.000    5.712    0.000 series.py:966(__getitem__)
   

In [31]:
from collections import defaultdict

def get_word_reviews_count_optimized(df):
    word_reviews_count = defaultdict(int)
    
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        
        unique_words = set(words)  # Уникальные слова в отзыве
        
        for word in unique_words:
            word_reviews_count[word] += 1
    
    return word_reviews_count

In [32]:
import line_profiler

profiler = line_profiler.LineProfiler(get_word_reviews_count_optimized, get_word_reviews_count)
profiler.enable()

word_reviews_count_optimized = get_word_reviews_count_optimized(reviews)
word_reviews_count = get_word_reviews_count(reviews)

profiler.disable()
profiler.print_stats()

Timer unit: 1e-09 s

Total time: 52.5688 s
File: <ipython-input-29-b1bc049bcd0c>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count(df):
     2         1       2709.0   2709.0      0.0      word_reviews = {}
     3    126679 14184707572.0 111973.6     27.0      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679 3295142692.0  26011.8      6.3          recipe_id, review = row['recipe_id'], row['review']
     5    126679  845634405.0   6675.4      1.6          words = review.split(' ')
     6   6792010 1873257435.0    275.8      3.6          for word in words:
     7   6617066 4105939066.0    620.5      7.8              if word not in word_reviews:
     8    174944  119418381.0    682.6      0.2                  word_reviews[word] = []
     9   6792010 5412698971.0    796.9     10.3              word_reviews[word].append(recipe_id)
    10

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [33]:
%%time
def calculate_mape_v1(df):
    total_mape = 0
    count = 0
    
    for _, row in df.iterrows():
        rating = row['rating']
        if rating != 0:
            average_rating = df[df['recipe_id'] == row['recipe_id']]['rating'].mean()
            mape = abs(rating - average_rating) / average_rating
            total_mape += mape
            count += 1
    
    if count > 0:
        return total_mape / count
    else:
        return None

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.7 µs


In [34]:
%%time
from numba import njit

@njit
def calculate_mape_v2(df):
    total_mape = 0
    count = 0
    
    for _, row in df.iterrows():
        rating = row['rating']
        if rating != 0:
            average_rating = df[df['recipe_id'] == row['recipe_id']]['rating'].mean()
            mape = abs(rating - average_rating) / average_rating
            total_mape += mape
            count += 1
    
    if count > 0:
        return total_mape / count
    else:
        return None

CPU times: user 291 ms, sys: 46.2 ms, total: 337 ms
Wall time: 1.1 s


In [35]:
%%time
import numpy as np

def calculate_mape_v3(df):
    ratings = df['rating'].values
    recipe_ids = df['recipe_id'].values
    
    mask = (ratings != 0)
    non_zero_ratings = ratings[mask]
    non_zero_recipe_ids = recipe_ids[mask]
    
    average_ratings = np.zeros_like(non_zero_ratings)
    unique_recipe_ids = np.unique(non_zero_recipe_ids)
    
    for recipe_id in unique_recipe_ids:
        mask = (non_zero_recipe_ids == recipe_id)
        average_ratings[mask] = np.mean(non_zero_ratings[mask])
    
    mape = np.abs(non_zero_ratings - average_ratings) / average_ratings
    
    return np.mean(mape)

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 14.8 µs


In [36]:
%%time
from numba import njit

@njit
def calculate_mape_v4(df):
    ratings = df['rating'].values
    recipe_ids = df['recipe_id'].values
    
    mask = (ratings != 0)
    non_zero_ratings = ratings[mask]
    non_zero_recipe_ids = recipe_ids[mask]
    
    average_ratings = np.zeros_like(non_zero_ratings)
    unique_recipe_ids = np.unique(non_zero_recipe_ids)
    
    for recipe_id in unique_recipe_ids:
        mask = (non_zero_recipe_ids == recipe_id)
        average_ratings[mask] = np.mean(non_zero_ratings[mask])
    
    mape = np.abs(non_zero_ratings - average_ratings) / average_ratings
    
    return np.mean(mape)

CPU times: user 383 µs, sys: 0 ns, total: 383 µs
Wall time: 387 µs


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4